# COVID Vaccine sense-checking

This notebook performs various checks on the vaccination table, filtered to COVID vaccines only. 

Checks include:
- Distinct vaccine names available 
- Possible values for schedule_part
- Range of dates for each vaccine
- Number of vaccines per patient

**Note: all row/patient counts are rounded to the nearest 10 and counts <=5 removed**


In [ ]:
## Import libraries

# %load_ext autoreload
# %autoreload 2

import pyodbc
import os
import pandas as pd
import numpy as np
from datetime import date, datetime
from IPython.display import display, Markdown

import sys
sys.path.append('../analysis/')
from utilities import *
from sense_checking import *

pd.set_option('display.max_colwidth', 250)

# get the server credentials from environ.txt
dbconn = os.environ.get('FULL_DATABASE_URL', None).strip('"')

In [ ]:
## Notebook run date

display(Markdown(f"""This notebook was run on {date.today().strftime('%Y-%m-%d')}.  The information below reflects the state of this dataset in OpenSAFELY-TPP as at this date."""))


In [ ]:
## Identify all COVID vaccines, from VaccinationReference table

vax_list_names = simple_sql(dbconn, table="VaccinationReference", col="VaccinationName_ID, VaccinationName", 
                       where="""LOWER(VaccinationContent) LIKE '%cov%' 
                                  or LOWER(VaccinationContent) LIKE '%coron%'
                                  or LOWER(VaccinationContent) LIKE '%sars%'""")
display(vax_list_names)
vax_list = [str(x) for x in vax_list_names["VaccinationName_ID"]]
vax_list_str = ','.join(vax_list)

In [ ]:
## Schema

table = "Vaccination"
where = {"": f"where vaccinationName_ID IN ({vax_list_str})"}

get_schema(dbconn, table, where)

In [ ]:
where = f"vaccinationName_ID IN ({vax_list_str})"
columns = ["VaccinationName", "VaccinationDate", "VaccinationSchedulePart"]
threshold = 50
    
counts_of_distinct_values(dbconn, table, columns, threshold=threshold, where=where, include_counts=True)

In [ ]:
## Dates per vaccine

columns = ["VaccinationDate"]
vaccines = vax_list
filters = [f"vaccinationName_ID = '{v}'" for v in vaccines]

for w in filters:
    counts_of_distinct_values(dbconn, table, columns=columns, threshold=3, where=w, sort_values=True) 


In [ ]:
## Patients with multiple records

counts_of_distinct_values(dbconn, table, columns=["patient_id"], threshold=50, where=where, frequency_count=True)  